In [3]:
import pandas as pd

input_filepath = "/Users/scottwillis/Documents/Dev/hiring-challenge-ml/docs_df/data.csv"

df = pd.read_csv(input_filepath)
df

,ID,Filename,Section,Subsection,Body
0,F7t1ajtf,JP_MHLW_MO_No.169-2004.csv,"Ordinance of the Ministry of Health, Labour an...",Business pertaining to services incidental to ...,一二三四五三二四五一二三
1,rnxIfAQE,JP_MHLW_MO_No.169-2004.csv,"Ordinance of the Ministry of Health, Labour an...",Business pertaining to services incidental to ...,一
2,9gWK1AVl,JP_MHLW_MO_No.169-2004.csv,"Ordinance of the Ministry of Health, Labour an...",Business pertaining to services incidental to ...,三
3,Eku39ikp,JP_MHLW_MO_No.169-2004.csv,"Ordinance of the Ministry of Health, Labour an...",Business pertaining to services incidental to ...,四
4,5kOY6Ne2,JP_MHLW_MO_No.169-2004.csv,"Ordinance of the Ministry of Health, Labour an...",Business pertaining to services incidental to ...,五
...,...,...,...,...,...
514,nZ/CnUtt,BR_Anforderungen_RDC_No.751-2022.csv,Ministry of Health - MSNational Health Surveil...,––––––––––––––––––––––––––––––––––––––,XXXXInstructions for use / User ManualXXXXGene...
515,+8pqL7/1,BR_Anforderungen_RDC_No.751-2022.csv,Ministry of Health - MSNational Health Surveil...,––––––––––––––––––––––––––––––––––––––,addresses)XXXXManufacturing Process (Flowchart...
516,WzGOVjRK,BR_Anforderungen_RDC_No.751-2022.csv,Ministry of Health - MSNational Health Surveil...,––––––––––––––––––––––––––––––––––––––,document issued by the International Medical D...
517,K8No5vRb,BR_Anforderungen_RDC_No.751-2022.csv,Ministry of Health - MSNational Health Surveil...,––––––––––––––––––––––––––––––––––––––,2) Applicable only when clinical evidence is r...


In [5]:
from req_analysis.util.openai import OpenAIClient
from dotenv import load_dotenv
import os
from tqdm import tqdm

load_dotenv()

oai_client = OpenAIClient(os.getenv("AZ_OPENAI_API_KEY"), os.getenv("OPENAI_API_BASE"))

system_prompt = """/
Classify the following information:

Filename: [filename]
Section: [section]
Subsection: [subsection]
Body: [body]

Please classify the following using the provided labels:

Type (choose one):
- requirement: A set regulatory requirement that has to be applied to the regulated system.
- note: A note or additional descriptions & examples which do not contain a requirement.
- definition: A definition of a term or concept.
- section_title: A title of a section, subsection, or chapter.

Subject Matter (choose one or more):
- process_qms: Processes or the Quality Management System as a whole
- materials: Materials
- sterilization: Sterilization
- maintenance: Maintenance
- software: Software
- usability: Usability
- risk_management: Risk Management
- clinical_evaluation: Clinical Evaluation
- biocompatibility: Biocompatibility
- packaging: Packaging
- labeling: Labeling
- registration: Registration
- testing: Testing
/"""

deployment_id = "GPT-4-TURBO"
output_filepath = "/Users/scottwillis/Documents/Dev/hiring-challenge-ml/docs_df/data_oai.csv"

df['Response'] = None

# Create a progress bar
progress_bar = tqdm(total=len(df), desc="Processing")

for index, row in df.iterrows():
    filename = row["Filename"]
    section = row["Section"]
    subsection = row["Subsection"]
    body = row["Body"]

    initial_prompt = f"""/
Filename: {filename}
Section: {section}
Subsection: {subsection}
Body: {body}
/"""

    messages = oai_client.make_messages(system_prompt, initial_prompt)
    response = oai_client.call_llm(messages, deployment_id)

    df.at[index, 'Response'] = response

    # Update the progress bar
    progress_bar.update(1)

    # Save the DataFrame to a CSV file
    df.to_csv(output_filepath, index=False)

# Close the progress bar
progress_bar.close()


Processing:   0%|          | 0/519 [00:00<?, ?it/s]

Processing:   8%|▊         | 40/519 [18:10<2:18:32, 17.35s/it]  